# Building with the Conversations API

This notebook presents practical patterns for managing persistent conversations using OpenAI's Conversations API. While the Responses API handles individual model calls, the Conversations API provides stateful conversation management - essential for building chat applications, multi-session agents, and systems requiring conversation history.

## What You'll Learn

- **Session management** for user conversations
- **Conversation templates** for common scenarios
- **Item management utilities** for efficient operations
- **Metadata tagging system** for organization
- **Conversation search and filtering** for retrieval
- **Export and import** for backup and portability
- **Conversation branching** for exploring alternatives
- **Pruning strategies** for managing long conversations
- **Analytics and insights** from conversation data
- **Integration with Responses API** for complete workflows

## Setup

In [ ]:
from openai import OpenAI
import os
import json
from typing import Optional, List, Dict, Any
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from collections import defaultdict

# Initialize the client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

## 1. Conversation Session Manager

When building multi-user applications, you need to manage conversation sessions efficiently. This pattern provides a complete session manager that handles creation, retrieval, lifecycle management, and cleanup - the foundation for any conversation-based application.

In [ ]:
@dataclass
class ConversationSession:
    """Represents a conversation session with metadata."""
    conversation_id: str
    user_id: str
    created_at: datetime
    last_active: datetime
    message_count: int = 0
    tags: List[str] = field(default_factory=list)
    custom_metadata: Dict[str, Any] = field(default_factory=dict)

class ConversationSessionManager:
    """Manage conversation sessions for multiple users."""
    
    def __init__(self, client: OpenAI, session_timeout: timedelta = timedelta(hours=24)):
        self.client = client
        self.session_timeout = session_timeout
        self.sessions: Dict[str, ConversationSession] = {}  # conversation_id -> session
        self.user_sessions: Dict[str, List[str]] = defaultdict(list)  # user_id -> [conversation_ids]
    
    def create_session(
        self,
        user_id: str,
        initial_message: Optional[str] = None,
        tags: Optional[List[str]] = None,
        **metadata
    ) -> ConversationSession:
        """Create a new conversation session."""
        # Prepare initial items
        items = []
        if initial_message:
            items.append({
                "type": "message",
                "role": "user",
                "content": initial_message
            })
        
        # Create conversation with metadata
        conversation = self.client.conversations.create(
            metadata={
                "user_id": user_id,
                "created_at": datetime.now().isoformat(),
                "tags": json.dumps(tags or []),
                **metadata
            },
            items=items if items else None
        )
        
        # Create session object
        session = ConversationSession(
            conversation_id=conversation.id,
            user_id=user_id,
            created_at=datetime.now(),
            last_active=datetime.now(),
            message_count=len(items),
            tags=tags or [],
            custom_metadata=metadata
        )
        
        # Store session
        self.sessions[conversation.id] = session
        self.user_sessions[user_id].append(conversation.id)
        
        return session
    
    def get_session(self, conversation_id: str) -> Optional[ConversationSession]:
        """Get a session by conversation ID."""
        session = self.sessions.get(conversation_id)
        
        if session and self._is_active(session):
            session.last_active = datetime.now()
            return session
        
        return None
    
    def get_user_sessions(
        self,
        user_id: str,
        active_only: bool = True
    ) -> List[ConversationSession]:
        """Get all sessions for a user."""
        conversation_ids = self.user_sessions.get(user_id, [])
        sessions = [self.sessions[cid] for cid in conversation_ids if cid in self.sessions]
        
        if active_only:
            sessions = [s for s in sessions if self._is_active(s)]
        
        return sorted(sessions, key=lambda s: s.last_active, reverse=True)
    
    def add_message(
        self,
        conversation_id: str,
        role: str,
        content: str
    ) -> None:
        """Add a message to a session."""
        session = self.get_session(conversation_id)
        if not session:
            raise ValueError(f"Session {conversation_id} not found or expired")
        
        # Add item to conversation
        self.client.conversations.items.create(
            conversation_id,
            items=[{
                "type": "message",
                "role": role,
                "content": [{"type": "input_text", "text": content}]
            }]
        )
        
        # Update session
        session.message_count += 1
        session.last_active = datetime.now()
    
    def close_session(self, conversation_id: str) -> None:
        """Close a session (mark as inactive)."""
        if conversation_id in self.sessions:
            # Update metadata to mark as closed
            session = self.sessions[conversation_id]
            self.client.conversations.update(
                conversation_id,
                metadata={
                    **session.custom_metadata,
                    "user_id": session.user_id,
                    "closed_at": datetime.now().isoformat(),
                    "status": "closed"
                }
            )
    
    def cleanup_expired_sessions(self) -> int:
        """Remove expired sessions. Returns count of cleaned up sessions."""
        expired = []
        
        for conv_id, session in self.sessions.items():
            if not self._is_active(session):
                expired.append(conv_id)
        
        # Remove from tracking
        for conv_id in expired:
            session = self.sessions[conv_id]
            if session.user_id in self.user_sessions:
                self.user_sessions[session.user_id].remove(conv_id)
            del self.sessions[conv_id]
        
        return len(expired)
    
    def _is_active(self, session: ConversationSession) -> bool:
        """Check if session is still active."""
        age = datetime.now() - session.last_active
        return age <= self.session_timeout
    
    def get_stats(self) -> Dict:
        """Get statistics about sessions."""
        active_sessions = [s for s in self.sessions.values() if self._is_active(s)]
        
        return {
            "total_sessions": len(self.sessions),
            "active_sessions": len(active_sessions),
            "unique_users": len(self.user_sessions),
            "total_messages": sum(s.message_count for s in self.sessions.values()),
            "avg_messages_per_session": sum(s.message_count for s in self.sessions.values()) / len(self.sessions) if self.sessions else 0
        }

# Example usage
manager = ConversationSessionManager(client, session_timeout=timedelta(hours=2))

# Create sessions for different users
session1 = manager.create_session(
    user_id="user_123",
    initial_message="I need help with Python",
    tags=["support", "python"],
    priority="high"
)
print(f"Created session: {session1.conversation_id}")

session2 = manager.create_session(
    user_id="user_456",
    initial_message="Tell me about machine learning",
    tags=["education", "ml"]
)
print(f"Created session: {session2.conversation_id}")

# Add messages to a session
manager.add_message(
    session1.conversation_id,
    role="assistant",
    content="I'd be happy to help! What specifically about Python?"
)

# Get user's sessions
user_sessions = manager.get_user_sessions("user_123")
print(f"\nUser 123 has {len(user_sessions)} active session(s)")

# Get statistics
stats = manager.get_stats()
print(f"\n📊 Statistics:")
print(f"   Total sessions: {stats['total_sessions']}")
print(f"   Active sessions: {stats['active_sessions']}")
print(f"   Unique users: {stats['unique_users']}")
print(f"   Total messages: {stats['total_messages']}")

## 2. Conversation Templates

Many applications need to start conversations with predefined structures or system prompts. This pattern provides a template system for creating conversations with consistent initial states - useful for customer support, onboarding flows, or specialized agents.

In [ ]:
@dataclass
class ConversationTemplate:
    """Template for creating conversations."""
    name: str
    description: str
    system_message: Optional[str] = None
    initial_messages: List[Dict[str, str]] = field(default_factory=list)
    default_metadata: Dict[str, Any] = field(default_factory=dict)
    tags: List[str] = field(default_factory=list)

class TemplateManager:
    """Manage conversation templates."""
    
    def __init__(self, client: OpenAI):
        self.client = client
        self.templates: Dict[str, ConversationTemplate] = {}
    
    def register_template(self, template: ConversationTemplate) -> None:
        """Register a template."""
        self.templates[template.name] = template
    
    def create_from_template(
        self,
        template_name: str,
        user_id: str,
        additional_metadata: Optional[Dict] = None,
        **variables
    ):
        """Create a conversation from a template."""
        if template_name not in self.templates:
            raise ValueError(f"Template '{template_name}' not found")
        
        template = self.templates[template_name]
        
        # Build items from template
        items = []
        
        # Add system message if present
        if template.system_message:
            system_msg = template.system_message.format(**variables)
            items.append({
                "type": "message",
                "role": "system",
                "content": system_msg
            })
        
        # Add initial messages
        for msg in template.initial_messages:
            content = msg["content"].format(**variables)
            items.append({
                "type": "message",
                "role": msg["role"],
                "content": content
            })
        
        # Merge metadata
        metadata = {
            **template.default_metadata,
            "user_id": user_id,
            "template": template_name,
            "tags": json.dumps(template.tags),
            **(additional_metadata or {})
        }
        
        # Create conversation
        conversation = self.client.conversations.create(
            metadata=metadata,
            items=items if items else None
        )
        
        return conversation
    
    def list_templates(self) -> List[str]:
        """List available templates."""
        return list(self.templates.keys())

# Create template manager
template_mgr = TemplateManager(client)

# Register templates
template_mgr.register_template(
    ConversationTemplate(
        name="customer_support",
        description="Customer support conversation",
        system_message="You are a helpful customer support agent for {company_name}. Be professional and empathetic.",
        initial_messages=[
            {
                "role": "assistant",
                "content": "Hello! Welcome to {company_name} support. How can I help you today?"
            }
        ],
        default_metadata={"category": "support", "priority": "normal"},
        tags=["support", "customer_service"]
    )
)

template_mgr.register_template(
    ConversationTemplate(
        name="coding_tutor",
        description="Programming education conversation",
        system_message="You are an expert {language} tutor. Explain concepts clearly with examples.",
        initial_messages=[
            {
                "role": "assistant",
                "content": "Hi! I'm your {language} tutor. What would you like to learn today?"
            }
        ],
        default_metadata={"category": "education"},
        tags=["education", "programming"]
    )
)

template_mgr.register_template(
    ConversationTemplate(
        name="onboarding",
        description="User onboarding flow",
        initial_messages=[
            {
                "role": "assistant",
                "content": "Welcome to our platform, {user_name}! Let's get you set up. First, what's your main goal?"
            }
        ],
        default_metadata={"category": "onboarding", "step": "1"},
        tags=["onboarding", "new_user"]
    )
)

# Example: Create conversations from templates
print("Available templates:", template_mgr.list_templates())

# Customer support conversation
support_conv = template_mgr.create_from_template(
    "customer_support",
    user_id="user_789",
    company_name="Acme Corp"
)
print(f"\n✓ Created support conversation: {support_conv.id}")

# Coding tutor conversation
tutor_conv = template_mgr.create_from_template(
    "coding_tutor",
    user_id="user_101",
    language="Python",
    additional_metadata={"skill_level": "beginner"}
)
print(f"✓ Created tutor conversation: {tutor_conv.id}")

# Onboarding conversation
onboard_conv = template_mgr.create_from_template(
    "onboarding",
    user_id="user_202",
    user_name="Alice"
)
print(f"✓ Created onboarding conversation: {onboard_conv.id}")

## 3. Batch Item Operations

Efficiently managing multiple conversation items is crucial for performance. This pattern provides utilities for batch operations like adding multiple messages, bulk deletions, and item transformations - essential when working with conversation history at scale.

In [ ]:
class ItemBatchOperations:
    """Utilities for batch operations on conversation items."""
    
    def __init__(self, client: OpenAI):
        self.client = client
    
    def add_messages_batch(
        self,
        conversation_id: str,
        messages: List[Dict[str, str]]
    ) -> List:
        """Add multiple messages in batches (max 20 per request)."""
        batch_size = 20
        all_results = []
        
        for i in range(0, len(messages), batch_size):
            batch = messages[i:i + batch_size]
            
            # Format items
            items = [
                {
                    "type": "message",
                    "role": msg["role"],
                    "content": [{"type": "input_text", "text": msg["content"]}]
                }
                for msg in batch
            ]
            
            # Add batch
            result = self.client.conversations.items.create(
                conversation_id,
                items=items
            )
            
            all_results.extend(result.data)
            print(f"Added batch {i//batch_size + 1}: {len(batch)} messages")
        
        return all_results
    
    def import_conversation_history(
        self,
        conversation_id: str,
        history: List[Dict[str, Any]]
    ) -> int:
        """Import conversation history from external format."""
        messages = [
            {"role": h["role"], "content": h["content"]}
            for h in history
        ]
        
        results = self.add_messages_batch(conversation_id, messages)
        return len(results)
    
    def get_all_items(
        self,
        conversation_id: str,
        limit_per_page: int = 100
    ) -> List:
        """Get all items from a conversation with pagination."""
        all_items = []
        after = None
        
        while True:
            # Fetch page
            page = self.client.conversations.items.list(
                conversation_id,
                limit=limit_per_page,
                after=after,
                order="asc"
            )
            
            all_items.extend(page.data)
            
            # Check if more pages exist
            if not page.has_more:
                break
            
            after = page.last_id
        
        return all_items
    
    def filter_items(
        self,
        conversation_id: str,
        role: Optional[str] = None,
        keyword: Optional[str] = None
    ) -> List:
        """Filter items by role or keyword."""
        items = self.get_all_items(conversation_id)
        filtered = []
        
        for item in items:
            # Filter by role
            if role and item.role != role:
                continue
            
            # Filter by keyword
            if keyword and hasattr(item, 'content'):
                text = ""
                for content_item in item.content:
                    if hasattr(content_item, 'text'):
                        text += content_item.text
                
                if keyword.lower() not in text.lower():
                    continue
            
            filtered.append(item)
        
        return filtered
    
    def get_conversation_summary(self, conversation_id: str) -> Dict:
        """Get summary statistics about a conversation."""
        items = self.get_all_items(conversation_id)
        
        role_counts = defaultdict(int)
        total_length = 0
        
        for item in items:
            role_counts[item.role] += 1
            
            if hasattr(item, 'content'):
                for content_item in item.content:
                    if hasattr(content_item, 'text'):
                        total_length += len(content_item.text)
        
        return {
            "total_items": len(items),
            "role_distribution": dict(role_counts),
            "total_characters": total_length,
            "avg_length_per_item": total_length / len(items) if items else 0
        }

# Example usage
batch_ops = ItemBatchOperations(client)

# Create a conversation for testing
test_conv = client.conversations.create(metadata={"purpose": "batch_test"})
print(f"Created conversation: {test_conv.id}")

# Import a conversation history
history = [
    {"role": "user", "content": "What is Python?"},
    {"role": "assistant", "content": "Python is a high-level programming language..."},
    {"role": "user", "content": "What are its key features?"},
    {"role": "assistant", "content": "Python's key features include simplicity, readability..."},
    {"role": "user", "content": "Can you show me an example?"},
    {"role": "assistant", "content": "Here's a simple Python example: print('Hello, World!')"},
]

imported_count = batch_ops.import_conversation_history(test_conv.id, history)
print(f"\n✓ Imported {imported_count} items")

# Get conversation summary
summary = batch_ops.get_conversation_summary(test_conv.id)
print(f"\n📊 Conversation Summary:")
print(f"   Total items: {summary['total_items']}")
print(f"   Role distribution: {summary['role_distribution']}")
print(f"   Total characters: {summary['total_characters']}")
print(f"   Avg length/item: {summary['avg_length_per_item']:.1f}")

# Filter items
user_messages = batch_ops.filter_items(test_conv.id, role="user")
print(f"\n🔍 Found {len(user_messages)} user messages")

python_mentions = batch_ops.filter_items(test_conv.id, keyword="Python")
print(f"🔍 Found {len(python_mentions)} items mentioning 'Python'")

## 4. Conversation Export and Import

Backing up conversations, migrating between systems, or analyzing conversation data requires export and import capabilities. This pattern provides utilities for serializing conversations to portable formats and restoring them - critical for data portability and backup strategies.

In [ ]:
import hashlib
from pathlib import Path

@dataclass
class ExportedConversation:
    """Exported conversation data."""
    conversation_id: str
    metadata: Dict[str, Any]
    items: List[Dict[str, Any]]
    exported_at: str
    version: str = "1.0"

class ConversationExporter:
    """Export and import conversations."""
    
    def __init__(self, client: OpenAI):
        self.client = client
    
    def export_conversation(
        self,
        conversation_id: str,
        include_system: bool = True
    ) -> ExportedConversation:
        """Export a conversation to a portable format."""
        # Get conversation metadata
        conversation = self.client.conversations.retrieve(conversation_id)
        
        # Get all items
        items = []
        after = None
        
        while True:
            page = self.client.conversations.items.list(
                conversation_id,
                limit=100,
                after=after,
                order="asc"
            )
            
            for item in page.data:
                # Skip system messages if not included
                if not include_system and item.role == "system":
                    continue
                
                # Extract item data
                item_data = {
                    "id": item.id,
                    "type": item.type,
                    "role": item.role,
                    "status": item.status
                }
                
                # Extract content
                if hasattr(item, 'content') and item.content:
                    content = []
                    for c in item.content:
                        if hasattr(c, 'text'):
                            content.append({"type": "text", "text": c.text})
                    item_data["content"] = content
                
                items.append(item_data)
            
            if not page.has_more:
                break
            
            after = page.last_id
        
        return ExportedConversation(
            conversation_id=conversation_id,
            metadata=conversation.metadata or {},
            items=items,
            exported_at=datetime.now().isoformat()
        )
    
    def save_to_file(
        self,
        exported: ExportedConversation,
        filepath: str
    ) -> None:
        """Save exported conversation to JSON file."""
        data = {
            "conversation_id": exported.conversation_id,
            "metadata": exported.metadata,
            "items": exported.items,
            "exported_at": exported.exported_at,
            "version": exported.version
        }
        
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
    
    def load_from_file(self, filepath: str) -> ExportedConversation:
        """Load exported conversation from JSON file."""
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        return ExportedConversation(**data)
    
    def import_conversation(
        self,
        exported: ExportedConversation,
        new_metadata: Optional[Dict] = None
    ):
        """Import a conversation from exported data."""
        # Merge metadata
        metadata = {
            **exported.metadata,
            "imported_at": datetime.now().isoformat(),
            "original_id": exported.conversation_id,
            **(new_metadata or {})
        }
        
        # Prepare items (max 20 for initial creation)
        initial_items = []
        for item in exported.items[:20]:
            if item["type"] == "message":
                content = item.get("content", [])
                if content:
                    text = content[0].get("text", "")
                    initial_items.append({
                        "type": "message",
                        "role": item["role"],
                        "content": text
                    })
        
        # Create conversation
        conversation = self.client.conversations.create(
            metadata=metadata,
            items=initial_items if initial_items else None
        )
        
        # Add remaining items if any
        if len(exported.items) > 20:
            remaining = exported.items[20:]
            batch_ops = ItemBatchOperations(self.client)
            
            messages = []
            for item in remaining:
                if item["type"] == "message":
                    content = item.get("content", [])
                    if content:
                        messages.append({
                            "role": item["role"],
                            "content": content[0].get("text", "")
                        })
            
            if messages:
                batch_ops.add_messages_batch(conversation.id, messages)
        
        return conversation

# Example usage
exporter = ConversationExporter(client)

# Export a conversation
print("Exporting conversation...")
exported = exporter.export_conversation(test_conv.id)
print(f"✓ Exported {len(exported.items)} items")
print(f"  Metadata: {exported.metadata}")
print(f"  Exported at: {exported.exported_at}")

# Save to file
filepath = "/tmp/conversation_export.json"
exporter.save_to_file(exported, filepath)
print(f"\n✓ Saved to: {filepath}")

# Load from file
loaded = exporter.load_from_file(filepath)
print(f"\n✓ Loaded conversation with {len(loaded.items)} items")

# Import as new conversation
imported_conv = exporter.import_conversation(
    loaded,
    new_metadata={"purpose": "imported_copy"}
)
print(f"\n✓ Imported as new conversation: {imported_conv.id}")

## 5. Conversation Branching

Sometimes you need to explore alternative conversation paths from a specific point. This pattern implements conversation branching - useful for A/B testing responses, exploring different scenarios, or implementing undo/redo functionality in chat interfaces.

In [ ]:
@dataclass
class Branch:
    """Represents a conversation branch."""
    branch_id: str
    parent_conversation_id: str
    branch_point: str  # Item ID where branch starts
    created_at: datetime
    description: str

class ConversationBranchManager:
    """Manage conversation branching."""
    
    def __init__(self, client: OpenAI):
        self.client = client
        self.branches: Dict[str, List[Branch]] = defaultdict(list)
    
    def create_branch(
        self,
        conversation_id: str,
        branch_point_item_id: Optional[str] = None,
        description: str = "Branch"
    ):
        """Create a branch from a conversation at a specific point."""
        # Get items up to branch point
        items = []
        after = None
        include_remaining = branch_point_item_id is None
        
        while True:
            page = self.client.conversations.items.list(
                conversation_id,
                limit=100,
                after=after,
                order="asc"
            )
            
            for item in page.data:
                if include_remaining:
                    items.append(item)
                elif item.id == branch_point_item_id:
                    items.append(item)
                    include_remaining = True
                elif not include_remaining:
                    items.append(item)
            
            if not page.has_more or (branch_point_item_id and include_remaining):
                break
            
            after = page.last_id
        
        # Create new conversation with items
        formatted_items = []
        for item in items[:20]:  # Max 20 for creation
            if item.type == "message":
                content_text = ""
                if hasattr(item, 'content'):
                    for c in item.content:
                        if hasattr(c, 'text'):
                            content_text = c.text
                            break
                
                formatted_items.append({
                    "type": "message",
                    "role": item.role,
                    "content": content_text
                })
        
        # Get original metadata
        original = self.client.conversations.retrieve(conversation_id)
        
        # Create branch
        branch_conv = self.client.conversations.create(
            metadata={
                **(original.metadata or {}),
                "parent_conversation": conversation_id,
                "branch_point": branch_point_item_id or "end",
                "branch_description": description,
                "is_branch": "true"
            },
            items=formatted_items if formatted_items else None
        )
        
        # Add remaining items if needed
        if len(items) > 20:
            batch_ops = ItemBatchOperations(self.client)
            messages = []
            
            for item in items[20:]:
                if item.type == "message":
                    content_text = ""
                    if hasattr(item, 'content'):
                        for c in item.content:
                            if hasattr(c, 'text'):
                                content_text = c.text
                                break
                    
                    messages.append({
                        "role": item.role,
                        "content": content_text
                    })
            
            if messages:
                batch_ops.add_messages_batch(branch_conv.id, messages)
        
        # Track branch
        branch = Branch(
            branch_id=branch_conv.id,
            parent_conversation_id=conversation_id,
            branch_point=branch_point_item_id or "end",
            created_at=datetime.now(),
            description=description
        )
        self.branches[conversation_id].append(branch)
        
        return branch_conv
    
    def get_branches(self, conversation_id: str) -> List[Branch]:
        """Get all branches of a conversation."""
        return self.branches.get(conversation_id, [])
    
    def compare_branches(
        self,
        branch_id_1: str,
        branch_id_2: str
    ) -> Dict:
        """Compare two branches."""
        batch_ops = ItemBatchOperations(self.client)
        
        items_1 = batch_ops.get_all_items(branch_id_1)
        items_2 = batch_ops.get_all_items(branch_id_2)
        
        return {
            "branch_1_items": len(items_1),
            "branch_2_items": len(items_2),
            "difference": abs(len(items_1) - len(items_2))
        }

# Example usage
branch_mgr = ConversationBranchManager(client)

# Create a branch from existing conversation
print("Creating branch from conversation...")
branch1 = branch_mgr.create_branch(
    test_conv.id,
    description="Alternative response path"
)
print(f"✓ Created branch: {branch1.id}")

# Add different content to the branch
client.conversations.items.create(
    branch1.id,
    items=[{
        "type": "message",
        "role": "assistant",
        "content": [{"type": "input_text", "text": "This is an alternative response in the branch."}]
    }]
)
print("✓ Added unique content to branch")

# Create another branch
branch2 = branch_mgr.create_branch(
    test_conv.id,
    description="Another alternative"
)
print(f"\n✓ Created second branch: {branch2.id}")

# Get all branches
branches = branch_mgr.get_branches(test_conv.id)
print(f"\n📊 Total branches of conversation: {len(branches)}")
for i, branch in enumerate(branches, 1):
    print(f"   {i}. {branch.description} (ID: {branch.branch_id})")

# Compare branches
comparison = branch_mgr.compare_branches(branch1.id, branch2.id)
print(f"\n🔍 Branch comparison:")
print(f"   Branch 1: {comparison['branch_1_items']} items")
print(f"   Branch 2: {comparison['branch_2_items']} items")
print(f"   Difference: {comparison['difference']} items")

## 6. Conversation Analytics

Understanding conversation patterns helps improve your application. This pattern provides analytics utilities for extracting insights from conversations - useful for identifying common issues, measuring engagement, and optimizing your agent's performance.

In [ ]:
from collections import Counter
import re

class ConversationAnalytics:
    """Analytics tools for conversations."""
    
    def __init__(self, client: OpenAI):
        self.client = client
    
    def analyze_conversation(self, conversation_id: str) -> Dict:
        """Comprehensive analysis of a conversation."""
        # Get all items
        items = []
        after = None
        
        while True:
            page = self.client.conversations.items.list(
                conversation_id,
                limit=100,
                after=after
            )
            items.extend(page.data)
            
            if not page.has_more:
                break
            after = page.last_id
        
        # Analyze
        role_counts = Counter()
        message_lengths = []
        all_text = []
        
        for item in items:
            role_counts[item.role] += 1
            
            if hasattr(item, 'content'):
                for content in item.content:
                    if hasattr(content, 'text'):
                        text = content.text
                        message_lengths.append(len(text))
                        all_text.append(text)
        
        # Calculate metrics
        avg_length = sum(message_lengths) / len(message_lengths) if message_lengths else 0
        
        # Calculate turn-taking
        turns = 0
        last_role = None
        for item in items:
            if item.role != last_role:
                turns += 1
                last_role = item.role
        
        return {
            "total_messages": len(items),
            "role_distribution": dict(role_counts),
            "avg_message_length": avg_length,
            "total_characters": sum(message_lengths),
            "conversation_turns": turns,
            "messages_per_role": {
                role: count / len(items) * 100
                for role, count in role_counts.items()
            }
        }
    
    def extract_keywords(
        self,
        conversation_id: str,
        top_n: int = 10
    ) -> List[tuple]:
        """Extract most common keywords from conversation."""
        # Get all text
        all_text = []
        after = None
        
        while True:
            page = self.client.conversations.items.list(
                conversation_id,
                limit=100,
                after=after
            )
            
            for item in page.data:
                if hasattr(item, 'content'):
                    for content in item.content:
                        if hasattr(content, 'text'):
                            all_text.append(content.text)
            
            if not page.has_more:
                break
            after = page.last_id
        
        # Extract words (simple tokenization)
        text = " ".join(all_text).lower()
        words = re.findall(r'\b[a-z]{3,}\b', text)  # Words 3+ chars
        
        # Remove common stop words
        stop_words = {'the', 'and', 'for', 'are', 'but', 'not', 'you', 'with', 'this', 'that', 'from', 'have', 'can', 'will', 'what'}
        words = [w for w in words if w not in stop_words]
        
        # Count and return top N
        word_counts = Counter(words)
        return word_counts.most_common(top_n)
    
    def calculate_engagement(
        self,
        conversation_id: str
    ) -> Dict:
        """Calculate engagement metrics."""
        analysis = self.analyze_conversation(conversation_id)
        
        # Simple engagement score based on:
        # - Number of turns
        # - Average message length
        # - Role balance
        
        turns = analysis['conversation_turns']
        avg_length = analysis['avg_message_length']
        role_dist = analysis.get('messages_per_role', {})
        
        # Calculate balance (closer to 50/50 is better)
        user_pct = role_dist.get('user', 0)
        balance_score = 100 - abs(50 - user_pct) * 2
        
        # Normalize scores
        turn_score = min(turns * 10, 100)  # Cap at 100
        length_score = min(avg_length / 5, 100)  # Cap at 100
        
        # Overall engagement
        engagement_score = (turn_score + length_score + balance_score) / 3
        
        return {
            "engagement_score": engagement_score,
            "turn_score": turn_score,
            "length_score": length_score,
            "balance_score": balance_score,
            "rating": self._rate_engagement(engagement_score)
        }
    
    def _rate_engagement(self, score: float) -> str:
        """Rate engagement level."""
        if score >= 80:
            return "Excellent"
        elif score >= 60:
            return "Good"
        elif score >= 40:
            return "Fair"
        else:
            return "Low"

# Example usage
analytics = ConversationAnalytics(client)

# Analyze conversation
print("Analyzing conversation...")
analysis = analytics.analyze_conversation(test_conv.id)

print(f"\n📊 Conversation Analysis:")
print(f"   Total messages: {analysis['total_messages']}")
print(f"   Role distribution: {analysis['role_distribution']}")
print(f"   Avg message length: {analysis['avg_message_length']:.1f} chars")
print(f"   Total characters: {analysis['total_characters']}")
print(f"   Conversation turns: {analysis['conversation_turns']}")

# Extract keywords
keywords = analytics.extract_keywords(test_conv.id, top_n=5)
print(f"\n🔑 Top Keywords:")
for word, count in keywords:
    print(f"   {word}: {count} times")

# Calculate engagement
engagement = analytics.calculate_engagement(test_conv.id)
print(f"\n💬 Engagement Metrics:")
print(f"   Overall score: {engagement['engagement_score']:.1f}/100")
print(f"   Rating: {engagement['rating']}")
print(f"   Turn score: {engagement['turn_score']:.1f}")
print(f"   Length score: {engagement['length_score']:.1f}")
print(f"   Balance score: {engagement['balance_score']:.1f}")

## 7. Integration with Responses API

The real power comes from combining Conversations and Responses APIs. This pattern shows how to seamlessly integrate both - creating a complete workflow for building stateful agents that generate responses while maintaining conversation history.

In [ ]:
class ConversationalAgent:
    """Agent that integrates Conversations and Responses APIs."""
    
    def __init__(
        self,
        client: OpenAI,
        model: str = "gpt-5-mini",
        instructions: Optional[str] = None
    ):
        self.client = client
        self.model = model
        self.instructions = instructions
        self.conversation_id: Optional[str] = None
    
    def start_conversation(self, user_id: str, **metadata) -> str:
        """Start a new conversation."""
        conversation = self.client.conversations.create(
            metadata={
                "user_id": user_id,
                "started_at": datetime.now().isoformat(),
                **metadata
            }
        )
        self.conversation_id = conversation.id
        return conversation.id
    
    def send_message(
        self,
        user_message: str,
        **response_kwargs
    ) -> str:
        """Send a message and get response."""
        if not self.conversation_id:
            raise ValueError("No active conversation. Call start_conversation first.")
        
        # Add user message to conversation
        self.client.conversations.items.create(
            self.conversation_id,
            items=[{
                "type": "message",
                "role": "user",
                "content": [{"type": "input_text", "text": user_message}]
            }]
        )
        
        # Generate response using Responses API with conversation context
        response = self.client.responses.create(
            model=self.model,
            conversation=self.conversation_id,
            input=user_message,
            instructions=self.instructions,
            **response_kwargs
        )
        
        # Add assistant response to conversation
        self.client.conversations.items.create(
            self.conversation_id,
            items=[{
                "type": "message",
                "role": "assistant",
                "content": [{"type": "input_text", "text": response.output_text}]
            }]
        )
        
        return response.output_text
    
    def get_history(self) -> List[Dict[str, str]]:
        """Get conversation history."""
        if not self.conversation_id:
            return []
        
        items = []
        after = None
        
        while True:
            page = self.client.conversations.items.list(
                self.conversation_id,
                limit=100,
                after=after,
                order="asc"
            )
            
            for item in page.data:
                if item.role in ["user", "assistant"]:
                    content_text = ""
                    if hasattr(item, 'content'):
                        for c in item.content:
                            if hasattr(c, 'text'):
                                content_text = c.text
                                break
                    
                    items.append({
                        "role": item.role,
                        "content": content_text
                    })
            
            if not page.has_more:
                break
            after = page.last_id
        
        return items
    
    def end_conversation(self) -> None:
        """End the current conversation."""
        if self.conversation_id:
            self.client.conversations.update(
                self.conversation_id,
                metadata={
                    "ended_at": datetime.now().isoformat(),
                    "status": "completed"
                }
            )
            self.conversation_id = None

# Example usage: Build a complete conversational agent
agent = ConversationalAgent(
    client,
    model="gpt-5-mini",
    instructions="You are a helpful assistant. Be concise and friendly."
)

# Start conversation
conv_id = agent.start_conversation(
    user_id="user_999",
    category="general_chat"
)
print(f"Started conversation: {conv_id}")

# Have a conversation
messages = [
    "Hello! Can you help me?",
    "What's the difference between Python and JavaScript?",
    "Which one is better for beginners?"
]

for user_msg in messages:
    print(f"\n👤 User: {user_msg}")
    response = agent.send_message(user_msg, max_output_tokens=150)
    print(f"🤖 Agent: {response[:100]}...")

# Get full history
history = agent.get_history()
print(f"\n📜 Conversation History ({len(history)} messages)")

# End conversation
agent.end_conversation()
print(f"\n✓ Conversation ended")

## Summary

This notebook presented essential patterns for building with the Conversations API:

1. **Session Management** - Multi-user conversation tracking and lifecycle
2. **Conversation Templates** - Reusable conversation structures
3. **Batch Operations** - Efficient item management
4. **Export & Import** - Data portability and backup
5. **Conversation Branching** - Exploring alternative paths
6. **Analytics** - Insights from conversation data
7. **API Integration** - Combining Conversations and Responses APIs

These patterns provide the foundation for building production-ready conversational applications. The Conversations API handles state management while the Responses API generates intelligent outputs - together they enable sophisticated multi-turn agents with persistent memory.